In [1]:
import datetime
import numpy as np
import os
import pandas as pd
import shutil
import urllib.request
import zipfile
import time
from functools import wraps
from math import trunc
from sklearn.metrics import root_mean_squared_error
from numba import jit

In [4]:
# VARIANTS object defines data configurations for different file sizes.
# Each key represents the data size (e.g., "100k" for 100 thousand).
VARIANTS = {
    "100k": {"filename": "u.data", "sep": "\t"},
    "1m": {"filename": "ratings.dat", "sep": r"::"},
    "20m": {"filename": "ratings.csv", "sep": ","},
}

In [5]:
# Define the chosen data variant (e.g., "100k", "1m", or "20m")
variant = "20m"

In [6]:
# Check if the chosen variant is a valid key in the VARIANTS object
if variant not in VARIANTS:
    # If not valid, raise an error
    raise ValueError(
        f"Invalid variant: {variant}. Valid options are {list(VARIANTS.keys())}"
    )

In [7]:
# Construct the URL for downloading the data based on the chosen variant
url = f"http://files.grouplens.org/datasets/movielens/ml-{variant}.zip"

In [8]:
# Extract information from the VARIANTS object for the chosen variant
variant_info = VARIANTS[variant]

# Destructure filename property from the variant information
filename = variant_info["filename"]

# Construct the directory name based on the variant
dirname = f"ml-{variant}"

# Construct the path to the downloaded zip file
zip_path = os.path.join(dirname + ".zip")

# NHO REMOVE CELL BEN DUOI

In [43]:
proxy_url = "http://26.26.26.1:10809"

proxy_support = urllib.request.ProxyHandler({"http": proxy_url})

opener = urllib.request.build_opener(proxy_support)

urllib.request.install_opener(opener)

In [44]:
# Download the data from the URL and save it to the zip_path
with urllib.request.urlopen(url) as r, open(zip_path, "wb") as f:
    # Copy the downloaded data from the response to the file
    shutil.copyfileobj(r, f)

In [45]:
# Extract the data from the downloaded zip file
with zipfile.ZipFile(zip_path, "r") as zf:
    # Extract all files from the zip archive to the current directory
    zf.extractall()

In [46]:
# Remove zip file after extraction (optional)
os.remove(zip_path)

In [9]:
# Construct the path to the CSV file
csv_path = os.path.join(dirname, filename)

# Define the column names for the data
names = ["u_id", "i_id", "rating", "timestamp"]

# Define data type for each column
dtype = {"u_id": np.uint32, "i_id": np.uint32, "rating": np.float64}

In [10]:
# Read the CSV data into a Pandas DataFrame
df = pd.read_csv(
    csv_path,
    names=names,
    dtype=dtype,
    header=0,
    sep=VARIANTS[variant]["sep"],
)

In [11]:
# Drop the "timestamp" column from the DataFrame
df.drop("timestamp", inplace=True, axis=1)

# Sort the DataFrame by the "u_id" column (assuming user IDs)
df.sort_values(by="u_id", inplace=True)

# Reset the index after sorting (optional, keeps row numbers aligned with data)
df.reset_index(drop=True, inplace=True)

In [12]:
# Randomly sample 80% for training set
train = df.sample(frac=0.8, random_state=7)

# Sample 50% from remaining for validation
val = df.drop(train.index.tolist()).sample(frac=0.5, random_state=8)

# Remaining data becomes test set
test = df.drop(train.index.tolist()).drop(val.index.tolist())

In [13]:
@jit(cache=True)
def shuffle(X):
    np.random.shuffle(X)
    return X

In [14]:
@jit(cache=True)
def initialization(n_users, n_items, n_factors):
    # Initialize user biases with zeros (n_users length)
    bu = np.zeros(n_users)

    # Initialize item biases with zeros (n_items length)
    bi = np.zeros(n_items)

    # Random user factors (normal distribution, mean 0, std 0.1)
    pu = np.random.normal(0, 0.1, (n_users, n_factors))

    # Random item factors (normal distribution, mean 0, std 0.1)
    qi = np.random.normal(0, 0.1, (n_items, n_factors))

    return bu, bi, pu, qi

In [15]:
@jit(cache=True)
def run_epoch(X, bu, bi, pu, qi, global_mean, n_factors, lr, reg):
    # Loop through each rating in the data matrix X
    for i in range(X.shape[0]):
        # Extract user ID, item ID, and rating from current row
        user, item, rating = int(X[i, 0]), int(X[i, 1]), X[i, 2]

        # Predict current rating
        pred = global_mean + bu[user] + bi[item]
        for factor in range(n_factors):
            pred += pu[user, factor] * qi[item, factor]

        # Calculate the error between predicted and actual rating
        err = rating - pred

        # Update user and item biases with learning rate (lr) and regularization (reg)
        bu[user] += lr * (err - reg * bu[user])
        bi[item] += lr * (err - reg * bi[item])

        # Update user and item latent factors for all factors (n_factors)
        for factor in range(n_factors):
            puf = pu[user, factor]
            qif = qi[item, factor]

            pu[user, factor] += lr * (err * qif - reg * puf)
            qi[item, factor] += lr * (err * puf - reg * qif)

    return bu, bi, pu, qi


In [16]:
@jit(cache=True)
def compute_val_metrics(X_val, bu, bi, pu, qi, global_mean, n_factors):
    residuals = []

    for i in range(X_val.shape[0]):
        user, item, rating = int(X_val[i, 0]), int(X_val[i, 1]), X_val[i, 2]
        pred = global_mean

        if user > -1:
            pred += bu[user]

        if item > -1:
            pred += bi[item]

        if (user > -1) and (item > -1):
            for factor in range(n_factors):
                pred += pu[user, factor] * qi[item, factor]

        residuals.append(rating - pred)

    residuals = np.array(residuals)
    loss = np.square(residuals).mean()
    rmse = np.sqrt(loss)
    mae = np.absolute(residuals).mean()

    return loss, rmse, mae

In [17]:
class bcolors:
    HEADER = "\033[95m"
    OKBLUE = "\033[94m"
    OKCYAN = "\033[96m"
    OKGREEN = "\033[92m"
    WARNING = "\033[93m"
    FAIL = "\033[91m"
    ENDC = "\033[0m"
    BOLD = "\033[1m"
    UNDERLINE = "\033[4m"

In [18]:
class SVD:
    def __init__(
        self,
        lr=0.005,
        reg=0.02,
        n_epochs=20,
        n_factors=100,
        early_stopping=True,
        shuffle=True,
        min_delta=0.001,
        min_rating=1,
        max_rating=5,
    ):

        self.lr = lr
        self.reg = reg
        self.n_epochs = n_epochs
        self.n_factors = n_factors
        self.early_stopping = early_stopping
        self.shuffle = shuffle
        self.min_delta = min_delta
        self.min_rating = min_rating
        self.max_rating = max_rating

    def fit(self, X, X_val=None):
        # Preprocess the training data
        X = self.preprocess_data(X)

        # Check if validation data is provided
        if X_val is not None:
            # Preprocess the validation data
            X_val = self.preprocess_data(X_val, train=False)
            # Initialize metrics for evaluation
            self.init_metrics()

        # Calculate the global mean of the rating feature
        self.global_mean_ = np.mean(X[:, 2])

        # Run the Stochastic Gradient Descent (SGD) algorithm
        self.run_sgd(X, X_val)

        return self

    def preprocess_data(self, X, train=True):
        # Copy the data to avoid modifying the original DataFrame
        X = X.copy()

        # Mappings are only needed during training
        if train:
            # Create unique mappings for user and item IDs (if training)
            user_ids = X["u_id"].unique().tolist()
            item_ids = X["i_id"].unique().tolist()

            n_users = len(user_ids)
            n_items = len(item_ids)

            user_idx = range(n_users)
            item_idx = range(n_items)

            self.user_mapping_ = dict(zip(user_ids, user_idx))
            self.item_mapping_ = dict(zip(item_ids, item_idx))

        # Replace user and item IDs with their corresponding indices in the mappings
        X["u_id"] = X["u_id"].map(self.user_mapping_)
        X["i_id"] = X["i_id"].map(self.item_mapping_)

        # Tag unseen users/items in validation data with -1 (for handling unknown entries)
        X.fillna(-1, inplace=True)

        X["u_id"] = X["u_id"].astype(np.int32)
        X["i_id"] = X["i_id"].astype(np.int32)

        return X[["u_id", "i_id", "rating"]].values

    def init_metrics(self):
        # Initialize empty metrics array with zeros
        metrics = np.zeros((self.n_epochs, 3), dtype=float)

        # Create a pandas DataFrame from the metrics array
        self.metrics_ = pd.DataFrame(metrics, columns=["Loss", "RMSE", "MAE"])

    def run_sgd(self, X, X_val):
        # Get number of unique users
        n_users = len(np.unique(X[:, 0]))

        # Get number of unique items
        n_items = len(np.unique(X[:, 1]))

        # Initialize model parameters
        bu, bi, pu, qi = initialization(n_users, n_items, self.n_factors)

        # Run SGD for specified number of epochs
        for epoch_ix in range(self.n_epochs):
            start = self.on_epoch_begin(epoch_ix)

            if self.shuffle:
                X = shuffle(X)

            # Update model parameters using run_epoch function
            bu, bi, pu, qi = run_epoch(
                X, bu, bi, pu, qi, self.global_mean_, self.n_factors, self.lr, self.reg
            )

            if X_val is not None:
                # Compute validation metrics if validation data provided
                self.metrics_.loc[epoch_ix, :] = compute_val_metrics(
                    X_val, bu, bi, pu, qi, self.global_mean_, self.n_factors
                )
                self.on_epoch_end(
                    start,
                    self.metrics_.loc[epoch_ix, "Loss"],
                    self.metrics_.loc[epoch_ix, "RMSE"],
                    self.metrics_.loc[epoch_ix, "MAE"],
                )

                if self.early_stopping:
                    val_rmse = self.metrics_["RMSE"].tolist()
                    if self.check_early_stopping(val_rmse, epoch_ix, self.min_delta):
                        break

            else:
                self.on_epoch_end(start)

        # Update internal model parameters with learned values
        self.bu_ = bu
        self.bi_ = bi
        self.pu_ = pu
        self.qi_ = qi

    def predict(self, X, clip=True):
        # Generate predictions for each user-item pair in X
        return [
            self.predict_pair(u_id, i_id, clip)
            for u_id, i_id in zip(X["u_id"], X["i_id"])
        ]

    def predict_pair(self, u_id, i_id, clip=True):
        # Initialize flags indicating if user and item are known
        user_known, item_known = False, False

        # Start prediction with global mean rating
        pred = self.global_mean_

        # Check if user ID exists in user mapping
        if u_id in self.user_mapping_:
            user_known = True
            # Get user index
            u_ix = self.user_mapping_[u_id]
            # Add user bias
            pred += self.bu_[u_ix]

        # Check if item ID exists in item mapping
        if i_id in self.item_mapping_:
            item_known = True
            # Get user index
            i_ix = self.item_mapping_[i_id]
            # Add user bias
            pred += self.bi_[i_ix]

        # If both user and item are known, add user-item interaction component
        if user_known and item_known:
            pred += np.dot(self.pu_[u_ix], self.qi_[i_ix])

        # Clip the predicted rating to the defined range (if clip is True)
        if clip:
            pred = self.max_rating if pred > self.max_rating else pred
            pred = self.min_rating if pred < self.min_rating else pred

        return pred

    def check_early_stopping(self, val_rmse, epoch_idx, min_delta):
        if epoch_idx > 0:
            # Check if validation RMSE has worsened by more than min_delta
            if val_rmse[epoch_idx] + min_delta > val_rmse[epoch_idx - 1]:
                # Update metrics DataFrame up to the current epoch
                self.metrics_ = self.metrics_.loc[: (epoch_idx + 1), :]
                return True
        return False

    def on_epoch_begin(self, epoch_ix):
        start = time.time()
        end = "  | " if epoch_ix < 9 else " | "
        print(
            f"{bcolors.WARNING}[EPOCH]:{bcolors.ENDC} {epoch_ix + 1}/{self.n_epochs}",
            end=end,
        )

        return start

    def on_epoch_end(self, start, val_loss=None, val_rmse=None, val_mae=None):
        end = time.time()

        if val_loss is not None:
            print(f"[VAL_LOSS]: {val_loss:.2f}", end=" - ")
            print(f"[VAL_RMSE]: {val_rmse:.2f}", end=" - ")
            print(f"[VAL_MAE]: {val_mae:.2f}", end=" | ")

        print(f"{bcolors.OKGREEN}[TIME]:{bcolors.ENDC} {end - start:.2f} s")

In [19]:
svd = SVD(
    lr=0.001,
    reg=0.005,
    n_epochs=100,
    n_factors=15,
    early_stopping=True,
    shuffle=True,
    min_rating=1,
    max_rating=5,
)

start = time.time()
svd.fit(X=train, X_val=val)
end = time.time()

print(f"\n{bcolors.FAIL}[PROCESSING TIME]:{bcolors.ENDC} {end - start:.2f} s")

[EPOCH]: 1/100  | [VAL_LOSS]: 0.84 - [VAL_RMSE]: 0.92 - [VAL_MAE]: 0.71 | [TIME]: 5.40 s
[EPOCH]: 2/100  | [VAL_LOSS]: 0.81 - [VAL_RMSE]: 0.90 - [VAL_MAE]: 0.69 | [TIME]: 5.29 s
[EPOCH]: 3/100  | [VAL_LOSS]: 0.79 - [VAL_RMSE]: 0.89 - [VAL_MAE]: 0.69 | [TIME]: 5.35 s
[EPOCH]: 4/100  | [VAL_LOSS]: 0.78 - [VAL_RMSE]: 0.88 - [VAL_MAE]: 0.68 | [TIME]: 5.29 s
[EPOCH]: 5/100  | [VAL_LOSS]: 0.77 - [VAL_RMSE]: 0.88 - [VAL_MAE]: 0.68 | [TIME]: 5.29 s
[EPOCH]: 6/100  | [VAL_LOSS]: 0.77 - [VAL_RMSE]: 0.87 - [VAL_MAE]: 0.67 | [TIME]: 5.26 s
[EPOCH]: 7/100  | [VAL_LOSS]: 0.76 - [VAL_RMSE]: 0.87 - [VAL_MAE]: 0.67 | [TIME]: 5.27 s
[EPOCH]: 8/100  | [VAL_LOSS]: 0.76 - [VAL_RMSE]: 0.87 - [VAL_MAE]: 0.67 | [TIME]: 5.26 s
[EPOCH]: 9/100  | [VAL_LOSS]: 0.76 - [VAL_RMSE]: 0.87 - [VAL_MAE]: 0.67 | [TIME]: 5.27 s
[EPOCH]: 10/100 | [VAL_LOSS]: 0.75 - [VAL_RMSE]: 0.87 - [VAL_MAE]: 0.67 | [TIME]: 5.26 s
[EPOCH]: 11/100 | [VAL_LOSS]: 0.75 - [VAL_RMSE]: 0.87 - [VAL_MAE]: 0.67 | [TIME]: 5.32 s
[EPOCH]: 12/100 | [VA

In [20]:
pred = svd.predict(test)

In [21]:
rmse = root_mean_squared_error(test["rating"], pred)
print(f"Test RMSE: {rmse:.2f}")

Test RMSE: 0.80
